In [ ]:
import pandas as pd
from pyexpat.errors import messages
from tqdm import tqdm

from src.util import LanguageModel

tqdm.pandas()

In [ ]:
df = pd.read_csv('data/combined_articles_with_sentiment.csv', index_col=0)
#df.rename(columns={df.columns[0]: 'index'}, inplace=True)
def clean(df_uncleaned):
    df_uncleaned = df_uncleaned[df_uncleaned['source_id'] != '1']
    return df_uncleaned
df = clean(df)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Amount of articles we the crawling failed
len(df[df['full_article'] == '<ERROR: ArticleException>']) / len(df)

In [ ]:
df_errors = df[df['full_article'] == '<ERROR: ArticleException>']

In [ ]:
# Step 1: Total entries per source_name in the original df
total_counts = df['source_name'].value_counts()

# Step 2: Error counts per source_name in df_errors
error_counts = df_errors['source_name'].value_counts()

# Step 3: Proportion of errors for each source_name
error_proportions = (error_counts / total_counts).fillna(0)  # Fill NaN with 0 for sources without errors

# Convert to a DataFrame for better visualization
error_proportions_df = error_proportions.reset_index()
error_proportions_df.columns = ['source_name', 'error_proportion']

print(error_proportions_df)


In [2]:
import pathlib
import pandas as pd
from pyexpat.errors import messages
from tqdm import tqdm



df = pd.read_csv(pathlib.Path("data") / "subset4_articles_with_sentiment_and_classification.csv", index_col=0)

In [3]:
df.head()

,id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,...,full_article,textblob_polarity,textblob_subjectivity,vader_neg,vader_neu,vader_pos,vader_compound,label,explanation,retrieved_context
0,287,the-irish-times,The Irish Times,Andrew Pulver,Kristen Stewart: I was told to tone down my se...,The Twilight actor says she was advised to sto...,https://www.irishtimes.com/culture/film/kriste...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T12:58:03Z,Kristen Stewart was told to stop holding her g...,...,Kristen Stewart was told to stop holding her g...,0.250598,0.488836,0.025,0.847,0.128,0.9769,True,"The article appears to be factual, as it quote...",['Permission (film) | Permission is a romantic...
1,8561,cbs-news,CBS News,Grace Segers,2020 Democrats reveal third quarter fundraisin...,The contenders in the 2020 presidential race h...,https://www.cbsnews.com/live-news/2020-democra...,https://cbsnews3.cbsistatic.com/hub/i/r/2019/0...,2019-10-01T14:29:16Z,"By Grace Segers, Sarah Ewall-Wice\r\nOctober 1...",...,"Booker says impeachment trial could be ""big bl...",0.000000,0.100000,0.000,1.000,0.000,0.0000,True,The article appears to be discussing the fundr...,"['Democratic Party presidential primaries, 202..."
2,1497,cnn,CNN,"Leah Asmelash, CNN",A Minnesota woman was killed in a rare black b...,A Minnesota woman was attacked and killed by a...,https://www.cnn.com/2019/09/05/us/minnesota-wo...,https://cdn.cnn.com/cnnnext/dam/assets/1909051...,2019-09-05T16:46:13Z,NaN,...,CNN —\n\nA Minnesota woman was attacked and ki...,-0.089000,0.392000,0.109,0.869,0.021,-0.9791,True,This article appears to be a factual report of...,"[""Night of the Grizzlies | Night of the Grizzl..."
3,10376,cnn,CNN,"David Williams, CNN",A cheerleader jumped off a homecoming float to...,Cheerleader Tyra Winters is Rockwall High Scho...,https://www.cnn.com/2019/10/03/us/cheerleader-...,https://cdn.cnn.com/cnnnext/dam/assets/1910031...,2019-10-03T14:51:32Z,NaN,...,CNN —\n\nCheerleader Tyra Winters is Rockwall ...,0.022465,0.448420,0.064,0.859,0.077,0.0102,True,The article is about a 17-year-old cheerleader...,"[""Karen Link | Link is a NFL Cheerleader for t..."
4,206,espn,ESPN,Adam RittenbergESPN Senior Writer,Taggart: Dehydration possible factor in FSU loss,Florida State will monitor the Seminoles' hydr...,http://espn.go.com/college-football/story/_/id...,https://a2.espncdn.com/combiner/i?img=%2Fphoto...,2019-09-03T15:16:00Z,After watching Florida State wear down in Satu...,...,Robert Mahone runs in for the easy 1-yard touc...,0.076696,0.403968,0.043,0.846,0.111,0.9713,True,The article reports on a real event in which F...,"[""2016 South Florida Bulls football team | The..."


In [4]:
df['label'].value_counts()

label
True     6134
False     185
Name: count, dtype: int64

In [5]:
df.iloc[0]['retrieved_context']

'[\'Permission (film) | Permission is a romantic dramedy film written and directed by Brian Crano. The film stars Rebecca Hall as a woman on the brink of a marriage proposal from her boyfriend (Dan Stevens), but is impeded by the suggestion of her brother (David Joseph Craig) and his life partner (Morgan Spector) to "test date" other men before she ultimately settles down. Meanwhile, the film also follows the relationship of the gay couple as they decide whether or not to become parents.\', \'Georgia Brooks | Georgia Brooks is a fictional character from the Australian soap opera "Neighbours", played by Saskia Hampele. The actress was travelling through Kakadu with her boyfriend when she received a call from her agent about a regular role on "Neighbours". Hampele was then asked to film an audition for the show and she said that she could not turn down the opportunity to play Georgia. She also believed that she was meant to get the part because the character was originally called Daisy, 

In [48]:
fake_news = df[df['label'] == False]

In [101]:
index = 21

In [102]:
fake_news.iloc[index]['title']

"NOAA issues statement supporting Trump's claim Hurricane Dorian threatened Alabama"

In [103]:
fake_news.iloc[index]['description']

'Five days after President Donald Trump tweeted that Hurricane Dorian threatened Alabama, NOAA weighed in.'

In [104]:
fake_news.iloc[index]['full_article']

'The agency issued its comment five days after the president\'s tweet.\n\nFive days after Donald Trump said Hurricane Dorian threatened Alabama, the National Oceanic and Atmospheric Association issued a statement late Friday supporting the president\'s claim and chastising a local branch of the National Weather Service.\n\nNOAA said the NWS Birmingham office was wrong to speak "in absolute terms" on Sunday when it tweeted: "Alabama will NOT see any impacts from #Dorian."\n\nThat tweet, at 11:11 a.m. EDT, followed 20 minutes after Trump\'s.\n\nA NOAA spokesperson said: "From Wednesday, August 28, through Monday, September 2, the information provided by NOAA and the National Hurricane Center to President Trump and the wider public demonstrated that tropical-storm-force winds from Hurricane Dorian could impact Alabama. This is clearly demonstrated in Hurricane Advisories #15 through #41, which can be viewed at the following link. The Birmingham National Weather Service\'s Sunday morning t

In [105]:
fake_news.iloc[index]['explanation']

"NOAA's statement supporting Trump's claim that Hurricane Dorian threatened Alabama is based on the fact that there was a 5% to 10% probability of a tiny part of southeastern Alabama experiencing winds of at least 39 mph. However, this probability was not unique to Alabama, as the same probabilities applied to the entire eastern seaboard. The statement from NOAA appears to be a technical justification rather than a clear indication that Alabama was significantly threatened by the hurricane. Considering the context of Trump's previous false claims, such as the wiretapping allegations and voter fraud claims, it's essential to approach this situation with a critical eye."

In [106]:
fake_news.iloc[index]['retrieved_context']

'[\'NAACP v. Trump | NAACP v. Trump, No. 1:17-cv-05427-ALC (S.D.N.Y. 2017), is a case pending before the United States District Court for the Southern District of New York. The plaintiffs, the National Association for the Advancement of Colored People (NAACP) and The Ordinary People Society allege that the defendants, President Donald Trump, the Vice President Michael Pence, and Kris Kobach are in violation of the Fifth and Fifteenth Amendments and the Federal Advisory Committee Act by establishing the Presidential Advisory Commission on Election Integrity(PEIC) for the purpose of supporting the President’s claim that "Voter fraud is all too common and then they criticize us for saying that ... take a look at Philadelphia, what\\\'s going on, take a look at Chicago, take a look at St. Louis."\', "Trump Tower wiretapping allegations | On March 4, 2017, U.S. President Donald Trump wrote a series of posts on his Twitter account that accused former President Barack Obama of wiretapping his

In [107]:
outlet_data = pd.DataFrame({
    "source_name": [
        "Reuters", "The Irish Times", "Al Jazeera English", "BBC News", "ABC News",
        "The New York Times", "CNN", "CBS News", "Newsweek", "Business Insider",
        "The Wall Street Journal", "ESPN"
    ],
    "leaning": [
        "Center", "NA", "Left-Center", "Center", "Left-Center",
        "Left-Center", "Left", "Left-Center", "Center", "Center",
        "Center", "Left-Center"
    ]
})

In [108]:
import pandas as pd

def transform_leaning(dataframe, column_name):
    """
    Replaces 'Center' with 0, 'Left-Center' with -0.5, 'Left' with -1, and 'NA' with None in the specified column of the dataframe.

    :param dataframe: The pandas DataFrame containing the column to be transformed.
    :param column_name: The name of the column to apply the transformation.
    :return: A pandas DataFrame with the transformed column.
    """
    mapping = {"Center": 0, "Left-Center": -0.5, "Left": -1, "NA": None}
    dataframe[column_name] = dataframe[column_name].map(mapping)
    return dataframe

# Apply the transformation
outlet_data = transform_leaning(outlet_data, "leaning")

# Display the transformed DataFrame
print(outlet_data)

                source_name  leaning
0                   Reuters      0.0
1           The Irish Times      NaN
2        Al Jazeera English     -0.5
3                  BBC News      0.0
4                  ABC News     -0.5
5        The New York Times     -0.5
6                       CNN     -1.0
7                  CBS News     -0.5
8                  Newsweek      0.0
9          Business Insider      0.0
10  The Wall Street Journal      0.0
11                     ESPN     -0.5


In [109]:
df = df.merge(outlet_data, on='source_name')
df.head()

,id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,...,textblob_polarity,textblob_subjectivity,vader_neg,vader_neu,vader_pos,vader_compound,label,explanation,retrieved_context,leaning
0,287,the-irish-times,The Irish Times,Andrew Pulver,Kristen Stewart: I was told to tone down my se...,The Twilight actor says she was advised to sto...,https://www.irishtimes.com/culture/film/kriste...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T12:58:03Z,Kristen Stewart was told to stop holding her g...,...,0.250598,0.488836,0.025,0.847,0.128,0.9769,True,"The article appears to be factual, as it quote...",['Permission (film) | Permission is a romantic...,NaN
1,8561,cbs-news,CBS News,Grace Segers,2020 Democrats reveal third quarter fundraisin...,The contenders in the 2020 presidential race h...,https://www.cbsnews.com/live-news/2020-democra...,https://cbsnews3.cbsistatic.com/hub/i/r/2019/0...,2019-10-01T14:29:16Z,"By Grace Segers, Sarah Ewall-Wice\r\nOctober 1...",...,0.000000,0.100000,0.000,1.000,0.000,0.0000,True,The article appears to be discussing the fundr...,"['Democratic Party presidential primaries, 202...",-0.5
2,1497,cnn,CNN,"Leah Asmelash, CNN",A Minnesota woman was killed in a rare black b...,A Minnesota woman was attacked and killed by a...,https://www.cnn.com/2019/09/05/us/minnesota-wo...,https://cdn.cnn.com/cnnnext/dam/assets/1909051...,2019-09-05T16:46:13Z,NaN,...,-0.089000,0.392000,0.109,0.869,0.021,-0.9791,True,This article appears to be a factual report of...,"[""Night of the Grizzlies | Night of the Grizzl...",-1.0
3,10376,cnn,CNN,"David Williams, CNN",A cheerleader jumped off a homecoming float to...,Cheerleader Tyra Winters is Rockwall High Scho...,https://www.cnn.com/2019/10/03/us/cheerleader-...,https://cdn.cnn.com/cnnnext/dam/assets/1910031...,2019-10-03T14:51:32Z,NaN,...,0.022465,0.448420,0.064,0.859,0.077,0.0102,True,The article is about a 17-year-old cheerleader...,"[""Karen Link | Link is a NFL Cheerleader for t...",-1.0
4,206,espn,ESPN,Adam RittenbergESPN Senior Writer,Taggart: Dehydration possible factor in FSU loss,Florida State will monitor the Seminoles' hydr...,http://espn.go.com/college-football/story/_/id...,https://a2.espncdn.com/combiner/i?img=%2Fphoto...,2019-09-03T15:16:00Z,After watching Florida State wear down in Satu...,...,0.076696,0.403968,0.043,0.846,0.111,0.9713,True,The article reports on a real event in which F...,"[""2016 South Florida Bulls football team | The...",-0.5


In [110]:
df.dtypes

id                                   int64
source_id                           object
source_name                         object
author                              object
title                               object
description                         object
url                                 object
url_to_image                        object
published_at                        object
content                             object
top_article                        float64
engagement_reaction_count          float64
engagement_comment_count           float64
engagement_share_count             float64
engagement_comment_plugin_count    float64
full_article                        object
textblob_polarity                  float64
textblob_subjectivity              float64
vader_neg                          float64
vader_neu                          float64
vader_pos                          float64
vader_compound                     float64
label                               object
explanation

In [111]:
df = pd.read_csv(pathlib.Path("data") / "subset4_articles_with_sentiment_and_classification.csv", index_col=0)

In [112]:
df.head(10)['explanation']

0    The article appears to be factual, as it quote...
1    The article appears to be discussing the fundr...
2    This article appears to be a factual report of...
3    The article is about a 17-year-old cheerleader...
4    The article reports on a real event in which F...
5    The article reports that Izzy Englander's Mill...
6    The article appears to be based on an excerpt ...
7    The article reports on the enduring popularity...
8    The article appears to be real as it reports o...
9    This article appears to be a factual report on...
Name: explanation, dtype: object

In [113]:
df['label'].value_counts()

label
True     6134
False     185
Name: count, dtype: int64